# Pycorrelate examples and validation

This notebook show `pycorrelate` usage as well comparisons with other 
less efficient implementations.

In [ ]:
import numpy as np
import h5py

In [ ]:
# Tweak here matplotlib style
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.sans-serif'].insert(0, 'Arial')
mpl.rcParams['font.size'] = 14
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [ ]:
import pycorrelate as pyc

## Load Data

In [ ]:
url = 'http://files.figshare.com/2182601/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5'
pyc.download_file(url, save_dir='data')

In [ ]:
fname = './data/' + url.split('/')[-1]
h5 = h5py.File(fname)
unit = 12.5e-9

In [ ]:
num_ph = int(3e6)
t = h5['photon_data']['timestamps'][:num_ph][h5['photon_data']['detectors'][:num_ph] == 0]
u = h5['photon_data']['timestamps'][:num_ph][h5['photon_data']['detectors'][:num_ph] == 1]

In [ ]:
t.shape, u.shape, t[0], u[0]

In [ ]:
t.max()*unit, u.max()*unit

Timestamps need to be monotonic, let's test it:

In [ ]:
assert (np.diff(t) > 0).all()
assert (np.diff(u) > 0).all()

## Log-scale bins (base 10)

In [ ]:
# compute lags in sec. then convert to timestamps
bins = pyc.make_lags(-7, 0, 10) / unit

In [ ]:
G = pyc.pcorrelate(t, u, bins)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(bins*unit, np.hstack((G[:1], G)), drawstyle='steps-pre')
plt.xlabel('Time (s)')
#for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
plt.xlim(30e-9, 2)
plt.xscale('log')
plt.grid(True)
plt.grid(True, which='minor', lw=0.3)

## Log-scale bins (base 2)

In [ ]:
# compute lags in timestamp units
bins = pyc.make_lags(-1, 28, 1, base=2).astype('int')

In [ ]:
G = pyc.pcorrelate(t, u, bins)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(bins*unit, np.hstack((G[:1], G)), drawstyle='steps-pre')
plt.xscale('log')
plt.xlabel('Time (s)')
#for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
plt.grid(True); plt.grid(True, which='minor', lw=0.3)
plt.xlim(30e-9, 2)

## Multi-tau bins

In [ ]:
n_group = 4
bin_widths = []
for i in range(26):
    bin_widths += [2**i]*n_group
np.array(bin_widths)
bins = np.hstack(([0], np.cumsum(bin_widths)))

In [ ]:
G = pyc.pcorrelate(t, u, bins)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(bins*unit, np.hstack((G[:1], G)), drawstyle='steps-pre')
plt.xscale('log')
plt.xlabel('Time (s)')
#for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
plt.xlim(30e-9, 2)
plt.grid(True)
plt.grid(True, which='minor', lw=0.3)

## Test: comparison with np.histogram

For testing we use smaller input arrays:

In [ ]:
tt = t[:5000]
uu = u[:5000]

The same algorims can be expressed in numpy in an incredible simple way
using `np.histogram`:

In [ ]:
# compute lags in sec. then convert to timestamps
bins = pyc.make_lags(-7, 0, 10) / unit

In [ ]:
Y = np.zeros(bins.size - 1, dtype=np.int64)
for ti in tt:
    Yc, _ = np.histogram(uu - ti, bins=bins)
    Y += Yc
G = Y / np.diff(bins)

In [ ]:
assert (G == pyc.pcorrelate(tt, uu, bins)).all()

Test passed! Here we demonstrated that the logic of the algorithm
is implemented as described in the paper (and in the few lines of code above).

## Tests: comparison with np.correlate

The comparison with `np.correlate` is a little tricky.
First we need to bin our input to create timetraces that can be correlated
by linear convolution. For testing purposes, let's choice
some timetrace bins:

In [ ]:
binwidth = 50e-6
bins_tt = np.arange(0, tt.max()*unit, binwidth) / unit
bins_uu = np.arange(0, uu.max()*unit, binwidth) / unit

In [ ]:
bins_tt.max()*unit, bins_tt.size

In [ ]:
bins_uu.max()*unit, bins_uu.size

In [ ]:
tx, _ = np.histogram(tt, bins=bins_tt)
ux, _ = np.histogram(uu, bins=bins_uu)

plt.figure(figsize=(10, 6))
plt.plot(bins_tt[1:]*unit, tx)
plt.plot(bins_uu[1:]*unit, ux)
plt.xlabel('Time (s)')

The plots above are the two curves we are going to feed to
`np.correlate`:

In [ ]:
C = np.correlate(ux, tx, mode='full')

We need to trim the result to obtain a proper alignment with
the 0-time lag:

In [ ]:
Gn = C[tx.size-1:]  # trim to positive time lags

Now, we can check that both `numpy.correlate` and `pycorrelate.ucorrelate`
give the same result:

In [ ]:
Gu = pyc.ucorrelate(tx, ux)
assert (Gu == Gn).all()

Now, let's compute the correlation also with `pycorrelate.pcorrelate`:

In [ ]:
maxlag_sec = 3.9
lagbins = (np.arange(0, maxlag_sec, binwidth) / unit).astype('int64')

In [ ]:
Gp = pyc.pcorrelate(tt, uu, lagbins) * int(binwidth / unit)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
Gn_t = np.arange(1, Gn.size+1) * binwidth * 1e3
Gu_t = np.arange(1, Gu.size+1) * binwidth * 1e3
Gp_t = lagbins[1:] * unit * 1e3
plt.plot(Gn_t, Gn, alpha=1, lw=2, label='numpy.correlate') 
plt.plot(Gu_t, Gu, alpha=0.6, lw=2, label='numpy.sum')
plt.plot(Gp_t, Gp, alpha=0.7, lw=2, label='pycorrelate.correlate')
plt.xlabel('Time (ms)', fontsize='large')
plt.grid(True)
plt.xlim(30e-3, 500)
plt.xscale('log')
plt.title('pycorrelate.correlate vs numpy.correlate', fontsize='x-large')
plt.legend(loc='best', fontsize='x-large');

## Conclusion

- `numpy.correlate` and `pycorrelate.ucorrelate` give identical results,
  with the latter being much faster. Note that inputs are swapped in the 
  two functions.
  
- `pycorrelate.ucorrelate` and `pycorrelate.pcorrelate` agree when using 
  uniform time-lag bins.